<a href="https://colab.research.google.com/github/aswarth123/Federated_Learning_Parameter_Clustering/blob/main/FederatedLearning(SVM)_clustering_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fuzzy-c-means

     |████████████████████████████████| 10.1 MB 5.7 MB/s 
     |████████████████████████████████| 15.7 MB 59 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import numpy as np
import random
import copy
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle
import cv2
import sys
import os
from fcmeans import FCM

# All Functions

## SVM Class


In [ ]:
class SVM:

  def __init__(self, X_train, y_train, X_test, y_test, val=True, val_type='k_fold', k=5, opt='mini_batch_GD', batch_size = 30, n_iters=100, learning_rate=0.001, lambda_param=0.01):

    self.lr = learning_rate
    self.lambda_param = lambda_param
    self.n_iters = n_iters

    self.X_train = X_train
    self.y_train = y_train

    self.X_test = X_test
    self.y_test = y_test

    self.val = val
    self.val_type=val_type
    self.k=k

    self.opt = opt
    self.batch_size = batch_size

    self.w = np.array([])
    self.b = None

  def grad(self,x,y):
    if y * (np.dot(x, self.w) - self.b) >= 1:
      dw = self.lr * (2 * self.lambda_param * self.w)
      db = 0
    else:
      dw = self.lr * (2 * self.lambda_param * self.w - np.dot(x, y))
      db = self.lr * y

    return (dw,db)
  

  def stochastic_GD(self, X_train, y_train, X_val=None, y_val=None):
    n_samples, n_features = X_train.shape  
    y_ = np.where(y_train <= 0, -1, 1)
          
    if self.w.size == 0 and self.b is None :
      self.w = np.zeros(n_features)
      self.b = 0

    w_best = np.zeros(n_features)
    b_best = 0

    acc_list = [] 
    for i in range(0,self.n_iters):
      for idx, x_i in enumerate(X_train):
        dw,db = self.grad(x_i,y_[idx])
        self.w -= dw
        self.b -= db
    
      if i%10 == 0 and self.val:
        approx_w = np.dot(X_val, self.w) - self.b
        approx_w = np.sign(approx_w)
        res_w = np.where(approx_w<0, 0, approx_w)

        approx_w_best = np.dot(X_val, w_best) - b_best
        approx_w_best = np.sign(approx_w_best)
        res_w_best = np.where(approx_w_best<0, 0, approx_w_best)
            
        if (accuracy_score(y_val, res_w_best) < accuracy_score(y_val, res_w)):
          w_best = copy.deepcopy(self.w)
          b_best = copy.deepcopy(self.b)


  def batch_GD(self, X_train, y_train, X_val=None, y_val=None):
      n_samples, n_features = X_train.shape  
      y_ = np.where(y_train <= 0, -1, 1)
            
      if self.w.size == 0 and self.b is None :
        self.w = np.zeros(n_features)
        self.b = 0

      w_best = np.zeros(n_features)
      b_best = 0

      acc_list = [] 
      for i in range(0,self.n_iters):
        dw_sum=0
        db_sum=0
        for idx, x_i in enumerate(X_train):
          dw,db = self.grad(x_i,y_[idx])
          dw_sum+=dw
          db_sum+=db
        self.w -= (dw_sum/n_samples)
        self.b -= (db_sum/n_samples)
      
        if i%10 == 0 and self.val:
          approx_w = np.dot(X_val, self.w) - self.b
          approx_w = np.sign(approx_w)
          res_w = np.where(approx_w<0, 0, approx_w)

          approx_w_best = np.dot(X_val, w_best) - b_best
          approx_w_best = np.sign(approx_w_best)
          res_w_best = np.where(approx_w_best<0, 0, approx_w_best)
              
          if (accuracy_score(y_val, res_w_best) < accuracy_score(y_val, res_w)):
            w_best = copy.deepcopy(self.w)
            b_best = copy.deepcopy(self.b)


  def mini_batch_GD(self, X_train, y_train, X_val=None, y_val=None):
      n_samples, n_features = X_train.shape  
      y_ = np.where(y_train <= 0, -1, 1)
            
      if self.w.size == 0 and self.b is None :
        self.w = np.zeros(n_features)
        self.b = 0

      w_best = np.zeros(n_features)
      b_best = 0

      acc_list = [] 

      # print(self.n_iters)
      
      for i in range(0,self.n_iters):
        # print(i)
        dw_sum=0.0
        db_sum=0.0
        s=0
        for idx, x_i in enumerate(X_train):
          dw,db = self.grad(x_i,y_[idx])
          dw_sum+=dw
          db_sum+=db
          s += 1
          if s%self.batch_size==0:
            self.w -= (dw_sum/self.batch_size)
            self.b -= (db_sum/self.batch_size)
      
        if i%10 == 0 and self.val:
          approx_w = np.dot(X_val, self.w) - self.b
          approx_w = np.sign(approx_w)
          res_w = np.where(approx_w<0, 0, approx_w)

          approx_w_best = np.dot(X_val, w_best) - b_best
          approx_w_best = np.sign(approx_w_best)
          res_w_best = np.where(approx_w_best<0, 0, approx_w_best)
              
          if (accuracy_score(y_val, res_w_best) < accuracy_score(y_val, res_w)):
            w_best = copy.deepcopy(self.w)
            b_best = copy.deepcopy(self.b)


  def cross_validation(self, val_split):

    X_train = np.concatenate((self.X_train[0],self.X_train[1]),axis=0)
    y_train = np.concatenate((self.y_train[0],self.y_train[1]),axis=0)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_split, random_state=1, stratify=y_train)

    eval("self."+self.opt+"(X_train, y_train, X_val, y_val)")


  def k_fold_cross_validation(self):

    X = np.concatenate((self.X_train[0],self.X_train[1]),axis=0)
    y = np.concatenate((self.y_train[0],self.y_train[1]),axis=0)

    w_list = []
    b_list = []
    acc_list = []

    if self.w.size == 0 and self.b == None:
      w = np.zeros(self.X_train[0].shape[1])
      b = 0
    else:
      w = copy.deepcopy(self.w)
      b = self.b

    skf = StratifiedKFold(n_splits=self.k, shuffle=True)

    for train_index, val_index in skf.split(X,y):
      
      X_train, X_val = X[train_index], X[val_index]
      y_train, y_val = y[train_index], y[val_index]

      eval("self."+self.opt+"(X_train, y_train, X_val, y_val)")

      print(self.accuracy())
      w_list.append(self.w)
      b_list.append(self.b)

      test_w = np.dot(X_val, self.w) - self.b
      test_w = np.sign(test_w)
      res_val = np.where(test_w<0,0,test_w)

      acc_list.append(accuracy_score(y_val, res_val))
    
      self.w = copy.deepcopy(w)
      self.b = b

    self.w = copy.deepcopy(w_list[acc_list.index(max(acc_list))])
    self.b = b_list[acc_list.index(max(acc_list))]
  

  def fit(self):
    if self.val_type == 'k_fold' and self.val:
      self.k_fold_cross_validation()
    
    elif self.val_type == 'cross_val' and self.val:
      self.cross_validation(0.2)
    
    elif not self.val:
      X_train = np.concatenate((self.X_train[0],self.X_train[1]),axis=0)
      y_train = np.concatenate((self.y_train[0],self.y_train[1]),axis=0)
      X_train, y_train= shuffle(X_train, y_train)
      eval("self."+self.opt+"(X_train, y_train)")

  def predict(self):
     approx = np.dot(self.X_test, self.w) - self.b
     approx = np.sign(approx)
     return np.where(approx<0, 0, approx)

  def accuracy(self):
    return accuracy_score(self.y_test, self.predict())*100


## Federated Class

In [ ]:
class Federated_SVM:

  def __init__(self, n_clients=5, val=True, val_type='k_fold', k=5, opt='mini_batch_GD', batch_size = 30, learning_rate=0.001, lambda_param=0.01, n_iters=100):
    self.n_clients = n_clients
    self.learning_rate = learning_rate
    self.lambda_param = lambda_param
    self.n_iters = n_iters
    self.val = val
    self.val_type = val_type
    self.client_distribution = []
    self.k = k
    self.opt = opt
    self.batch_size = batch_size
    self.X_test = None
    self.y_test = None
    self.X_clients_test = None
    self.y_clients_test = None

  def create_clients(self, X_train, y_train, X_clients_test, y_clients_test, X_test, y_test):
    self.clients=[]
    for i in range(self.n_clients):
      self.client_distribution.append(X_train[i][0].shape[0] + X_train[i][1].shape[0])
      self.clients.append(SVM(X_train[i],y_train[i], np.concatenate(X_clients_test[i],axis=0), np.concatenate(y_clients_test[i],axis=0), self.val, self.val_type, self.k, self.opt, self.batch_size, self.n_iters, self.learning_rate, self.lambda_param))
    self.X_test = copy.deepcopy(X_test)
    self.y_test = copy.deepcopy(y_test)
  

  def average_aggregator(self, parameter_list):
    w = np.zeros(parameter_list[0].shape[0])
    b = 0
    for i in range(0,2*self.n_clients,2):
        w = np.add(w,parameter_list[i]*self.client_distribution[i//2]/sum(self.client_distribution))
        b = b + parameter_list[i+1]*self.client_distribution[i//2]/sum(self.client_distribution)
    return (w, b)

  def highest_aggregator(self, parameter_list):
    score = 0

    for i in range(0,self.n_clients):
      acc = self.clients[i].accuracy()
      if (acc > score):
        w = copy.deepcopy(self.clients[i].w)
        b = self.clients[i].b
        acc = score

    return (w,b)

  def random_aggregator(self,parameter_list):
    n = random.randint(0,self.n_clients)
    w = np.zeros(parameter_list[0].shape[0])
    b = 0
    for i in range(0,2*self.n_clients,2):
      if i//2 != n:
        w = np.add(w,parameter_list[i]*self.client_distribution[i//2]/sum(self.client_distribution))
        b = b + parameter_list[i+1]*self.client_distribution[i//2]/sum(self.client_distribution)
      else:
        continue
    return (w, b)

  def adaptive_scaling_aggregator(self, parameter_list):
      clus_param = parameter_list[0]
      for i in range(2,len(parameter_list),2):
        clus_param = np.row_stack((clus_param,parameter_list[i]))

      # print(clus_param)
      
      fcm = FCM(n_clusters=3)
      fcm.fit(clus_param)
      res = fcm.predict(clus_param)
      prob = fcm.soft_predict(clus_param)
      
      clus_weights=[]

      for i in range(0,3):
        sw = np.zeros(len(parameter_list[0]))
        sb=0
        for j in range(0,len(parameter_list),2):
          sw += parameter_list[j]*prob[j//2][i]
          sb += parameter_list[j+1]*prob[j//2][i]
        

        clus_weights.append(sw/np.sum(prob[:][i]))
        clus_weights.append(sb/np.sum(prob[:][i]))

      for i in range(0,self.n_clients):
        
        self.clients[i].w = copy.deepcopy(clus_weights[2*res[i]])
        self.clients[i].b = copy.deepcopy(clus_weights[2*res[i]+1])

      print(prob)
      if res[0]==res[3]:
        print("clus 1 true")
      else:
        print("clus 1 flase")

      if res[1]==res[4]:
        print("clus 2 true")
      else:
        print("clus 2 flase")
        print('global test acc',self.accuracy(w_best,b_best))
      
      return(self.average_aggregator(parameter_list))

    
      
  def fit(self, g_iters, aggregator):
    w_best = np.zeros(self.X_test.shape[1])
    b_best = 0
    for i in range(0,g_iters):
      print('global round',i+1)
      for j in range(0,self.n_clients):
        if i==0 or aggregator == self.adaptive_scaling_aggregator:
          self.clients[j].fit()
        else:
          self.clients[j].w = copy.deepcopy(w_agg)
          self.clients[j].b = copy.deepcopy(b_agg)
          self.clients[j].fit()
        print('client',j+1,self.clients[j].accuracy())          
      parameter_list = []
      for k in range(0,self.n_clients):
        parameter_list.append(self.clients[k].w)
        parameter_list.append(self.clients[k].b)
        # print("client",k,self.clients[k].w+[self.clients[k].b])
      
      w_agg, b_agg = aggregator(parameter_list)
      # print("agg",self.accuracy(w_agg,b_agg),"best",self.accuracy(w_best,b_best))
      if self.accuracy(w_agg,b_agg)>self.accuracy(w_best,b_best) or i==0:
        w_best=copy.deepcopy(w_agg)
        b_best=copy.deepcopy(b_agg)
      print('global test acc',self.accuracy(w_best,b_best))


  def predict(self,w,b):
     approx = np.dot(self.X_test, w) - b
     approx = np.sign(approx)
     return np.where(approx<0, 0, 1)
  
  def accuracy(self,w,b):
    return accuracy_score(self.y_test, self.predict(w,b))*100


## Utility functions

In [ ]:
def create_kmeans_clusters(X, Y, n_clusters = 3, random_state = 0):
  clusters = KMeans(n_clusters=n_clusters, random_state=random_state).fit_predict(X)
  result = []
  for i in range(n_clusters):
    result.append(X[clusters == i])
    result.append(Y[clusters == i])
  return tuple(result)  

In [ ]:
def get_clients(class1, class2, n_clusters = 3, n_clients = 5):

  clients_X = []
  clients_y = []

  clientsXtest = []
  clientsYtest = []

  X_test = []
  y_test = []

  clusters_1 = KMeans(n_clusters=n_clusters, random_state=0).fit_predict(class1)
  clusters_2 = KMeans(n_clusters=n_clusters, random_state=0).fit_predict(class2)

  for i in range(n_clusters):

    X_train0, X_test0, y_train0, y_test0 = train_test_split(class1[clusters_1 == i],np.zeros((class1[clusters_1 == i].shape[0],)),test_size=0.2)
    X_train1, X_test1, y_train1, y_test1 = train_test_split(class2[clusters_2 == i],np.ones((class2[clusters_2 == i].shape[0],)),test_size=0.2)

    clients_X.append([X_train0, X_train1])
    clients_y.append([y_train0, y_train1])

    clientsXtest.append([X_test0,X_test1])
    clientsYtest.append([y_test0,y_test1])

    X_test.extend([X_test0,X_test1])
    y_test.extend([y_test0,y_test1])

  for i in range(0,2):
    X_train0_ex, X_test0_ex, y_train0_ex, y_test0_ex = train_test_split(clients_X[i][0],clients_y[i][0],test_size=0.5)
    X_train1_ex, X_test1_ex, y_train1_ex, y_test1_ex = train_test_split(clients_X[i][1],clients_y[i][1],test_size=0.5)

    clients_X.append([X_train0_ex, X_train1_ex])
    clients_y.append([y_train0_ex, y_train1_ex])

    clients_X[i][0],clients_y[i][0] = copy.deepcopy(X_test0_ex), copy.deepcopy(y_test0_ex)
    clients_X[i][1],clients_y[i][1] = copy.deepcopy(X_test1_ex), copy.deepcopy(y_test1_ex)

    X_train0_exte, X_test0_exte, y_train0_exte, y_test0_exte = train_test_split(clientsXtest[i][0],clientsYtest[i][0],test_size=0.5)
    X_train1_exte, X_test1_exte, y_train1_exte, y_test1_exte = train_test_split(clientsXtest[i][1],clientsYtest[i][1],test_size=0.5)    

    clientsXtest.append([X_train0_exte, X_train1_exte])
    clientsYtest.append([y_train0_exte, y_train1_exte])

    clientsXtest[i][0],clientsYtest[i][0] = copy.deepcopy(X_test0_exte), copy.deepcopy(y_test0_exte)
    clientsXtest[i][1],clientsYtest[i][1] = copy.deepcopy(X_test1_exte), copy.deepcopy(y_test1_exte)

    X_test.extend([X_test0_exte,X_test1_exte])
    y_test.extend([y_test0_exte,y_test1_exte])

  X_test = np.concatenate(X_test,axis=0)
  y_test = np.concatenate(y_test,axis=0)

  return clients_X,clients_y,clientsXtest,clientsYtest,X_test,y_test

In [ ]:
def load_mnist_return_required_digits(n1, n2):
  # Loading the mnist dataset and concatenating train - test sets
  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
  x_total = np.concatenate((x_train, x_test), axis=0)
  y_total = np.concatenate((y_train, y_test), axis=0)

  # Normalizing and reshaping the data
  x_total = x_total/255 
  x_total = x_total.reshape(x_total.shape[0],784)

  x_n1 = x_total[y_total == n1]
  y_n1 = y_total[y_total == n1]

  x_n2 = x_total[y_total == n2]
  y_n2 = y_total[y_total == n2]

  return [(x_n1, y_n1),(x_n2, y_n2)]

In [ ]:
def get_total_from_clients(clients_X,clients_y):
  x_train0 = [i[0] for i in clients_X]
  x_train0 = np.concatenate(x_train0, axis=0)
  x_train1 = [i[1] for i in clients_X]
  x_train1 = np.concatenate(x_train1, axis=0)
  y_train0 = [i[0] for i in clients_y]
  y_train0 = np.concatenate(y_train0, axis=0)
  y_train1 = [i[1] for i in clients_y]
  y_train1 = np.concatenate(y_train1, axis=0)

  return ([x_train0,x_train1],[y_train0,y_train1])    

# MNIST 0 & 6


In [ ]:
mnist = load_mnist_return_required_digits(0,6)

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
clients_X,clients_y,clientsXtest,clientsYtest,X_test,y_test = get_clients(mnist[0][0], mnist[1][0], n_clusters = 3, n_clients = 5)

In [ ]:
xtrain_gl, ytrain_gl = get_total_from_clients(clients_X,clients_y)

In [ ]:
f_svm = Federated_SVM(n_clients = 5, val=False, n_iters=150,opt='stochastic_GD')
f_svm.create_clients(clients_X,clients_y,clientsXtest,clientsYtest,X_test,y_test)

In [ ]:
f_svm.fit(10,f_svm.adaptive_scaling_aggregator)

global round 1
client 1 98.91540130151843
client 2 99.79797979797979
client 3 99.76387249114522
client 4 99.13232104121475
client 5 99.59514170040485
[[7.04045049e-01 1.27255242e-01 1.68699709e-01]
 [4.11825324e-02 6.58187974e-02 8.92998670e-01]
 [5.52225408e-04 9.98362366e-01 1.08540906e-03]
 [8.57934757e-01 6.43358492e-02 7.77293941e-02]
 [4.36811344e-02 6.33365842e-02 8.92982281e-01]]
clus 1 true
clus 2 true
global test acc 97.71136241249326
global round 2
client 1 99.13232104121475
client 2 100.0
client 3 99.64580873671783
client 4 99.34924078091106
client 5 99.59514170040485
[[1.47661178e-01 1.12554391e-01 7.39784431e-01]
 [8.88446801e-01 6.72065571e-02 4.43466419e-02]
 [8.65746825e-04 9.98669211e-01 4.65042040e-04]
 [8.93453391e-02 7.24189103e-02 8.38235751e-01]
 [8.85774541e-01 6.52033499e-02 4.90221093e-02]]
clus 1 true
clus 2 true
global test acc 97.81906300484653
global round 3
client 1 98.69848156182212
client 2 100.0
client 3 99.64580873671783
client 4 99.56616052060737
cli

# MNIST 3 & 8

In [ ]:
mnist = load_mnist_return_required_digits(3,8)

In [ ]:
clients_X,clients_y,clientsXtest,clientsYtest,X_test,y_test = get_clients(mnist[0][0], mnist[1][0], n_clusters = 3, n_clients = 5)

In [ ]:
f_svm = Federated_SVM(n_clients = 5, val=False, n_iters=150,opt='stochastic_GD')
f_svm.create_clients(clients_X,clients_y,clientsXtest,clientsYtest,X_test,y_test)

In [ ]:
f_svm.fit(10,f_svm.adaptive_scaling_aggregator)

global round 1
client 1 95.87155963302753
client 2 98.77800407331976
client 3 100.0
client 4 98.61751152073732
client 5 99.18533604887983
[[1.12446027e-01 1.26728754e-01 7.60825218e-01]
 [8.63201292e-01 8.32037391e-02 5.35949692e-02]
 [1.38578296e-03 9.97729508e-01 8.84708755e-04]
 [9.18483000e-02 8.98701418e-02 8.18281558e-01]
 [8.52068807e-01 9.15654783e-02 5.63657149e-02]]
clus 1 true
clus 2 true
global test acc 95.32509403546482
global round 2
client 1 95.87155963302753
client 2 98.77800407331976
client 3 100.0
client 4 98.15668202764977
client 5 98.16700610997964
[[9.58674266e-02 7.97701508e-01 1.06431066e-01]
 [8.54825849e-01 5.55884673e-02 8.95856835e-02]
 [1.17585605e-03 7.16320159e-04 9.98107824e-01]
 [9.90076483e-02 8.05903985e-01 9.50883671e-02]
 [8.58247719e-01 5.29434839e-02 8.88087970e-02]]
clus 1 true
clus 2 true
global test acc 95.32509403546482
global round 3
client 1 95.87155963302753
client 2 98.98167006109979
client 3 100.0
client 4 98.15668202764977
client 5 98.370